In [1]:
import os
import pickle

from dotenv import load_dotenv
import openai
from openai import OpenAI, AzureOpenAI

In [2]:
load_dotenv()

True

In [3]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

### Add context to maintain conversation continuity.

In [4]:
def save_messages(messages: list[dict]):
    try:
        with open('./db.dat', 'wb') as f:
            db = {'data': messages}
            pickle.dump(db, f)
    except:
        print('Failed to save messages.')


def load_messages() -> list[dict]:
    try:
        with open('./db.dat', 'rb') as f:
            db = pickle.load(f)
            return db.get('data', [])
    except:
        print('Failed to load messages.')
        return []

In [5]:
messages = []
# messages = load_messages()

system_prompt = "請扮演一位智能助理，記住我告訴過你的資訊。"
messages.append({"role": "system", "content": system_prompt})
print(f'System: {system_prompt}')

while True:
    try:
        user_prompt = input("User: ")
        if not user_prompt.strip():
            break
        messages.append({"role": "user", "content": user_prompt})
        print(f'User: {user_prompt}')

        completion = client.chat.completions.create(
            model="gpt-35-turbo-120",
            messages=messages
        )
        assistant_prompt = completion.choices[0].message.content
        messages.append({"role": "assistant", "content": assistant_prompt})
        print(f'ChatGPT: {assistant_prompt}')

    except openai.APIError as err:
        print(f"Error: {err}")
        break

# save_messages(messages)

System: 請扮演一位智能助理，記住我告訴過你的資訊。
User: 你好，你可以叫我寶寶，我今年三歲。
ChatGPT: 你好寶寶！很高興再次見到你。你今年三歲，這是一個美好的年齡！我會記住這個重要的資訊，以便在需要的時候提醒你的年齡。有什麼我可以幫助你的嗎？
User: 我喜歡吃蛋糕，你有推薦的款式嗎？
ChatGPT: 當然！對於喜歡吃蛋糕的你，我可以給你一些建議。以下是一些常見且受歡迎的蛋糕款式：

1. 巧克力蛋糕：如果你喜歡濃郁的巧克力口味，這是一個經典的選擇。巧克力蛋糕可以配上巧克力奶油或雪糕，讓你的味蕾感到快樂。

2. 草莓蛋糕：如果你喜歡酸甜的口味，草莓蛋糕可能是你的首選。這種蛋糕常常裝飾得很漂亮，上面滿滿地堆滿了新鮮的草莓。

3. 檸檬蛋糕：對於喜歡清新口味的人來說，檸檬蛋糕是一個不錯的選擇。它有著令人振奮的柑橘風味，讓你感到舒爽。

4. 起士蛋糕：如果你喜歡柔滑濕潤的口感，起士蛋糕可能是你喜愛的款式。它有著濃郁的乳酪味道，可以加上水果醬或巧克力醬來增添口味。

這些只是一些蛋糕的例子，當然還有其他更多選擇。你可以根據自己的口味和喜好選擇你最喜歡的蛋糕款式。享受美味的蛋糕吧！如果你還有其他問題，我隨時為你服務。
User: 我應該放上幾歲的蠟燭呢？
ChatGPT: 根據你告訴我的資訊，你今年三歲，因此可以放上三根蠟燭來慶祝生日。每一根蠟燭代表一年的成長，所以放上對應的數量是一個很有意義的方式來慶祝你的年齡增長。希望你度過一個美好的生日！如果你有其他疑問，請隨時告訴我。


### Create a Chatbot with Gradio
Reference: https://www.gradio.app/guides/quickstart

In [7]:
# Reference: https://www.gradio.app/guides/creating-a-chatbot-fast#additional-inputs

import gradio as gr


def get_response(message, history, system_prompt, stream, temperature):
    messages = []
    messages.append({"role": "system", "content": system_prompt})
    for user_prompt, assistant_prompt in history:
        messages.append({"role": "user", "content": user_prompt})
        messages.append({"role": "assistant", "content": assistant_prompt})
    messages.append({"role": "user", "content": message})

    completion = client.chat.completions.create(
        model="gpt-35-turbo-120",
        messages=messages,
        stream=stream,
        temperature=temperature
    )

    if stream:
        response = ''
        for chunk_completion in completion:
            if chunk_completion.choices:
                response += chunk_completion.choices[0].delta.content or ''
                yield response
    else:
        response = completion.choices[0].message.content
        yield response


app = gr.ChatInterface(
    get_response,
    additional_inputs=[
        gr.Textbox("You are helpful AI.", label="System Prompt"),
        gr.Checkbox(label='Stream', value=True),
        gr.Slider(0, 2, value=0.8, label="Temperature")
    ]
)


app.queue().launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


### Create a Chatbot with LINE